# 深入理解现有检索机制
---

## 1. 环境准备
- 安装依赖包
- 设置镜像源

In [1]:
# 设置国内镜像源（加速下载）
import os

from langchain_core.output_parsers import StrOutputParser

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
# 安装依赖包（取消注释运行）
# !pip install langchain faiss-cpu huggingface-hub dashscope PyPDF2

## 2. 文档处理
- 加载PDF文件
- 检查加载结果

In [5]:
from langchain_community.document_loaders import PyPDFLoader

def load_documents(pdf_paths):
    """加载pdf文档并返回LangChain Document对象列表"""
    all_docs = []
    for path in pdf_paths:
        try:
            loader = PyPDFLoader(path)
            docs = loader.load()
            all_docs.extend(docs)
            print(f"成功加载：{path}（共{len(docs)}页）")
        except Exception as e:
            print(f"加载失败{path}：{str(e)}")
    return all_docs

# 示例使用
pdf_paths = ["pdf_China/中国产品质量法.pdf"]
processed_documents = load_documents(pdf_paths) # 原始文件
print(processed_documents)

成功加载：pdf_China/中国产品质量法.pdf（共10页）
[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-03-16T11:54:10+08:00', 'moddate': '2025-03-16T11:54:10+08:00', 'source': 'pdf_China/中国产品质量法.pdf', 'total_pages': 10, 'page': 0, 'page_label': '1'}, page_content='中华人民共和国产品质量法  \n \n（1993年2月22日第七届全国人民代表大会常务委员会第三十次会议通过  根据 2000年7\n月8日第九届全国人民代表大会常务委员会第十六次会议 《关于修改〈中华人民共和国产品\n质量法〉的决定》第一次修正   根据 2009年8月27日第十一届全国人民代表大会常务委员\n会第十次会议《关于修改部分法律的决定》第二次修正  根据 2018年12月29日第十三届全\n国人民代表大会常务委员会第七次会议《关于修改〈中华人民共和国产品质量法〉等五部法\n律的决定》第三次修正）  \n \n  目  录 \n \n  第一章  总  则 \n  第二章  产品质量的监督  \n  第三章  生产者、销售者的产品质量责任和义务  \n  第一节  生产者的产品质量责任和义务  \n  第二节  销售者的产品质量责任和义务  \n  第四章  损害赔偿  \n  第五章  罚  则 \n  第六章  附  则 \n \n  第一章  总  则 \n \n  第一条  为了加强对产品质量的监督管理，提高产品质量水平，明确产品质量责任，保\n护消费者的合法权益，维护社会经济秩序，制定本法。  \n \n  第二条  在中华人民共和国境内从事产品生产、销售活动，必须遵守本法。  \n \n  本法所称产品是指经过加工、制作，用于销售的产品。  \n \n  建设工程不适用本法规定；但是，建设工程使用的建筑材料、建筑构配件和设备，属于\n前款规定的产品范围的，适用本法规定。  \n \n  第三条  生产者、 销售者应当建立健全内部产品质量管

## 3. 文本分割与向量化
- 中文文本分割
- 构建向量数据库

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(docs):
    """执行文本分割"""
    if not docs:
        raise ValueError("输入文档列表为空")

    # 中文优化分割器
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,     # 每个文本块500字符
        chunk_overlap=100,  # 块间重叠100字符
        separators=["\n\n", "\n", "。", "！", "？"] # 中文分隔符
    )

    # 执行分割
    split_docs = text_splitter.split_documents(docs)
    print(f"分割为{len(split_docs)}个文本块")

    # 查看前2个分割样例
    for i, doc in enumerate(split_docs[:2]):
        print(f"\n块{i+1}:\n{doc.page_content[:200]}...")

    return split_docs

# 示例使用
split_decs = split_documents(processed_documents)

分割为26个文本块

块1:
中华人民共和国产品质量法  
 
（1993年2月22日第七届全国人民代表大会常务委员会第三十次会议通过  根据 2000年7
月8日第九届全国人民代表大会常务委员会第十六次会议 《关于修改〈中华人民共和国产品
质量法〉的决定》第一次修正   根据 2009年8月27日第十一届全国人民代表大会常务委员
会第十次会议《关于修改部分法律的决定》第二次修正  根据 2018年12月29日第十三届全
国人...

块2:
第一章  总  则 
 
  第一条  为了加强对产品质量的监督管理，提高产品质量水平，明确产品质量责任，保
护消费者的合法权益，维护社会经济秩序，制定本法。  
 
  第二条  在中华人民共和国境内从事产品生产、销售活动，必须遵守本法。  
 
  本法所称产品是指经过加工、制作，用于销售的产品。  
 
  建设工程不适用本法规定；但是，建设工程使用的建筑材料、建筑构配件和设备，属于
前款规...


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

def create_vector_store(split_docs):
    """创建并返回向量数据库"""
    if not split_docs:
        raise ValueError("无法用空文档创建向量库")

    embeddings = HuggingFaceEmbeddings(
        model_name="distiluse-base-multilingual-cased-v2"
    )

    vector_store = FAISS.from_documents(split_docs, embeddings)
    print(f"向量库已构建（包含{vector_store.index.ntotal}）个向量")

    return vector_store

# 示例使用
vector_store = create_vector_store(split_decs)

向量库已构建（包含26）个向量


## 4. 分析当前检索系统的组成

- **向量检索**：目前使用的是FAISS + HuggingFace嵌入模型(distiluse-base-multilingual-cased-v2)
- **文本分割**：RecursiveCharacterTextSplitter以500字符为单位进行分割
- **检索参数**：top-k=3的相似度检索

## 5. 基础实验与性能评估

In [9]:
import numpy as np
# 检索分析实验
def analyze_retrieval(query, vector_store, top_k=3):
    """深入分析实验结果"""
    # 获取原始检索结果
    docs = vector_store.similarity_search(query, k=top_k)

    # 计算相似度分数
    embeddings = vector_store.embedding_function # 返回嵌入模型对象
    query_embedding = embeddings.embed_query(query) # 调用嵌入模型对问题编码

    print(f"\n查询：'{query}'")
    print(f"检索到的{len(docs)}个文档：")

    for i, doc in enumerate(docs[:top_k]):
        doc_embedding = embeddings.embed_query(doc.page_content) # 调用嵌入模型对检索到的文本编码
        similarity = np.dot(query_embedding, doc_embedding) # 计算查询向量和文档向量之间的余弦相似度
        print(f"\n📄 文档{i+1} (相似度: {similarity:.4f}):")
        print(f"  来源: {doc.metadata.get('source', '未知')}")
        print(f"  内容: {doc.page_content[:50]}...")

# 示例使用
queries = [
    "产品质量监督抽查的规定",
    "产品缺陷的法律定义",
    "违反产品质量法的处罚措施"
]

for query in queries:
    analyze_retrieval(query, vector_store)



查询：'产品质量监督抽查的规定'
检索到的3个文档：

📄 文档1 (相似度: 0.3063):
  来源: pdf_China/中国产品质量法.pdf
  内容: 国家监督抽查的产品，地方不得另行重复抽查；上级监督抽查的产品，下级不得另行重
复抽查。  
 
  ...

📄 文档2 (相似度: 0.4011):
  来源: pdf_China/中国产品质量法.pdf
  内容: 品或者有其他严重质量问题的产品， 以及直接用于生产、 销售该项产品的原辅材料、 包装物、
生产工具，...

📄 文档3 (相似度: 0.3465):
  来源: pdf_China/中国产品质量法.pdf
  内容: 管理部门予以公告；公告后经复查仍不合格的，责令停业，限期整顿；整顿期满后经复查产
品质量仍不合格的，...

查询：'产品缺陷的法律定义'
检索到的3个文档：

📄 文档1 (相似度: 0.3029):
  来源: pdf_China/中国产品质量法.pdf
  内容: 或者应当知道其权益受到损害时起计算。  
 
  因产品存在缺陷造成损害要求赔偿的请求权， 在造成损...

📄 文档2 (相似度: 0.3299):
  来源: pdf_China/中国产品质量法.pdf
  内容: 成犯罪的，依法追究刑事责任。  
 
  第六十二条  服务业的经营者将本法第四十九条至第五十二条规...

📄 文档3 (相似度: 0.2651):
  来源: pdf_China/中国产品质量法.pdf
  内容: 第七十条  本法第四十九条至第五十七条、第六十条至第六十三条规定的行政处罚由市
场监督管理部门决定。...

查询：'违反产品质量法的处罚措施'
检索到的3个文档：

📄 文档1 (相似度: 0.3492):
  来源: pdf_China/中国产品质量法.pdf
  内容: 管理部门予以公告；公告后经复查仍不合格的，责令停业，限期整顿；整顿期满后经复查产
品质量仍不合格的，...

📄 文档2 (相似度: 0.3455):
  来源: pdf_China/中国产品质量法.pdf
  内容: 据证明其不知道该产品为禁止销售的产品并如实说明其进货来源的，可以从轻或者减轻处
罚。 
 
  第五...

📄 文档3 (相似度: 0.3252):
  来源: pdf_China